# **Chains**

**In LangChain, chains refer to sequences of actions that combine multiple components or tasks into a structured pipeline. They allow developers to build complex workflows by linking together various operations, such as querying a language model, processing its output, retrieving information from external sources, and interacting with APIs or databases**

> Chains are easily reusable components linked together.
>
> Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc., and provide a simple interface to this sequence.

----

## **LLMChain**

LLMChain combined a prompt template, LLM, and output parser into a class.

Some advantages of switching to the LCEL implementation are:

Clarity around contents and parameters. The legacy LLMChain contains a default output parser and other options.

Easier streaming. LLMChain only supports streaming via callbacks.
Easier access to raw message outputs if desired. LLMChain only exposes these via a parameter or via callback.

In [1]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

prompt = ChatPromptTemplate.from_messages([
    ('user', 'Tell me a joke about {input}')
])

Legacy_chain = LLMChain(
    llm=ChatGroq(api_key="You-API-Key",
                 model='llama-3.1-8b-instant',
                 temperature=0.2),
    prompt = prompt
)

C:\Users\Hasnain\AppData\Local\Temp\ipykernel_13128\1460545614.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  Legacy_chain = LLMChain(


In [3]:
legacy_result = Legacy_chain({"input": "car"})
legacy_result

{'input': 'car',
 'text': 'Why did the car go to the doctor?\n\nBecause it was feeling a little "car-sick"!'}

**`Note`** that LLMChain by default returned a dict containing both the input and the output from StrOutputParser, so to extract the output, you need to access the "text" key.



In [4]:
legacy_result['text']

'Why did the car go to the doctor?\n\nBecause it was feeling a little "car-sick"!'

In [5]:
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
    ('user', 'Tell me a joke about {input}')
])

chain = prompt | ChatGroq(api_key="You-API-Key",
                 model='llama-3.1-8b-instant',
                 temperature=0.2) | StrOutputParser()

In [6]:
chain.invoke({"input": "house"})

'Why did the house go to therapy? \n\nBecause it had a lot of "foundation" issues.'

If you'd like to mimic the `dict` packaging of input and output in `LLMChain`, you can use a RunnablePassthrough.assign like:

In [7]:
from langchain_core.runnables import RunnablePassthrough

outer_chain = RunnablePassthrough().assign(text=chain)

outer_chain.invoke({"input": "candy"})

{'input': 'candy',
 'text': 'Why did the lollipop go to the party? \n\nBecause it was a sucker for fun.'}

---
# **ConversationChain**
ConversationChain incorporated a memory of previous messages to sustain a stateful conversation.

Some advantages of switching to the Langgraph implementation are:

- Innate support for threads/separate sessions. To make this work with `ConversationChain`, you'd need to instantiate a separate memory class outside the chain.
- More explicit parameters. `ConversationChain` contains a hidden default prompt, which can cause confusion.
- Streaming support. `ConversationChain` only supports streaming via callbacks.

In [8]:
from langchain.chains import ConversationChain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

template = """
You are a pirate. Answer the following questions as best you can.
Chat history: {history}
Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

memory = ConversationBufferMemory()

chain = ConversationChain(
    llm=ChatGroq(api_key="You-API-Key",
                 model='llama-3.1-8b-instant',
                 temperature=0.2),
    memory=memory,
    prompt=prompt,
)

chain({"input": "I'm Lava, how are you?"})

C:\Users\Hasnain\AppData\Local\Temp\ipykernel_13128\4010190727.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\Hasnain\AppData\Local\Temp\ipykernel_13128\4010190727.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(


{'input': "I'm Lava, how are you?",
 'history': '',
 'response': 'Arrr, I be doin\' just fine, Lava me hearty! Me and me crew, the "Blackheart Gang", have been sailin\' the seven seas, plunderin\' the riches of the landlubbers. Me ship, the "Maverick\'s Revenge", be in top condition, ready to set sail fer the next adventure. What be bringin\' ye to these waters, matey?'}

In [9]:
chain({"input": "What is my name?"})

{'input': 'What is my name?',
 'history': 'Human: I\'m Lava, how are you?\nAI: Arrr, I be doin\' just fine, Lava me hearty! Me and me crew, the "Blackheart Gang", have been sailin\' the seven seas, plunderin\' the riches of the landlubbers. Me ship, the "Maverick\'s Revenge", be in top condition, ready to set sail fer the next adventure. What be bringin\' ye to these waters, matey?',
 'response': "Ye be askin' about yer name, eh? Well, matey, I be rememberin' ye said it yerself, just a moment ago. Ye be Lava, the scurvy dog!"}